In [39]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import ReLU

In [16]:
batch_size = 64
num_classes = 10
learning_rate= 1e-3
num_epochs= 10

In [56]:
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))

])


trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=4)


In [58]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Model 

In [79]:
class lenet5(nn.Module):
    """Some Information about MyModule"""
    def __init__(self, num_classes):
        super(lenet5, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

        self.fc1 = nn.Linear(in_features=400, out_features=120)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84)
        self.relu2 = nn.ReLU()
        self.fc = nn.Linear(84, num_classes)




    def forward(self, x):

        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc(out)

        return out


In [80]:
model = lenet5(num_classes).to(device)
model

lenet5(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (relu2): ReLU()
  (fc): Linear(in_features=84, out_features=10, bias=True)
)

In [81]:
cost = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

Training

In [82]:
total_step = len(trainloader)
total_step

938

In [84]:
for epoch in range(num_epochs):
    for i, (img, label) in enumerate(trainloader):
        img = img.to(device)
        label = label.to(device)


        output = model(img)
        loss = cost(output, label)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
        if (i+1) % 400 == 0:
            print(f"epoch {epoch+1}/{num_epochs}, step {i+1}/{total_step}, loss: {loss.item()}")

print('Finished Operation')

epoch 1/10, step 400/938, loss: 0.06976968795061111
epoch 1/10, step 800/938, loss: 0.1825031042098999
epoch 2/10, step 400/938, loss: 0.09121204167604446
epoch 2/10, step 800/938, loss: 0.15097296237945557
epoch 3/10, step 400/938, loss: 0.06870236247777939
epoch 3/10, step 800/938, loss: 0.005682880990207195
epoch 4/10, step 400/938, loss: 0.05251723900437355
epoch 4/10, step 800/938, loss: 0.042246174067258835
epoch 5/10, step 400/938, loss: 0.017511842772364616
epoch 5/10, step 800/938, loss: 0.06292644888162613
epoch 6/10, step 400/938, loss: 0.010066384449601173
epoch 6/10, step 800/938, loss: 0.013208141550421715
epoch 7/10, step 400/938, loss: 0.027321210131049156
epoch 7/10, step 800/938, loss: 0.040980059653520584
epoch 8/10, step 400/938, loss: 0.017043698579072952
epoch 8/10, step 800/938, loss: 0.019709134474396706
epoch 9/10, step 400/938, loss: 0.0028120120987296104
epoch 9/10, step 800/938, loss: 0.18047992885112762
epoch 10/10, step 400/938, loss: 0.004487953614443541


Test

In [85]:
with torch.no_grad():
    correct = 0
    total = 0

    for img, label in testloader:
        img = img.to(device)
        label = label.to(device)
        outputs = model(img)

        _, predicted = torch.max(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
    
    accuracy = 100 * correct / total
    print('The Accuracy over 10,000 test images is: ', accuracy)

The Accuracy over 10,000 test images is:  99.23
